<a href="https://colab.research.google.com/github/BuberDev/Machine_Learning/blob/main/MixtureofExperts_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.Instalacja biblioteki Hugging Face Transformers i PyTorch



In [1]:
!pip install transformers
!pip install torch
!pip install -q torch==1.0.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.0.0 (from versions: 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0)
ERROR: No matching distribution found for torch==1.0.0


## 2.Pobranie danych za pomocą polecenia curl:

In [2]:
import requests

url = "http://ipfs.opentensor.ai/api/v0/object/get?arg=Qme2dawBzozFGtKWX73fh5fmB8NJD7TRS2XSWKhJB4WbJd"
data = requests.get(url).text


## 3.Przetworzenie danych wejściowych do formatu, który jest zgodny z wejściem modelu GPT-Neo-2.7B:

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
inputs = tokenizer.encode(data, return_tensors="pt")


In [8]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


## 4.Definicja modelu MoE:

In [4]:
!pip install --upgrade transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:

!pip install tqdm boto3 requests regex sentencepiece sacremoses
from transformers import GPTNeoForCausalLM
from transformers.models.moe.moe_for_causal_lm import MoEForCausalLMConfig

config = MoEForCausalLMConfig(
    num_experts=8,
    gate_hidden_size=2048,
    expert_hidden_size=4096,
    num_groups=4,
    num_att_heads=16,
    moe_layer_norm=True,
    dropout=0.1,
)

model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B", config=config)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.1/135.1 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 4.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=237e28137c54a4744c2c0c676685801889a8dbd744d577f25d83fd45d1598515
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


ModuleNotFoundError: ignored

## 5.Ustawienie hiperparametrów treningu i kompilacja modelu:

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup

batch_size = 2
learning_rate = 5e-5
num_epochs = 3

optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=len(inputs) * num_epochs // batch_size
)


## 6. Przygotowanie danych do treningu:

In [ ]:
dataset = TensorDataset(inputs)
train_dataloader = DataLoader(
    dataset, sampler=RandomSampler(dataset), batch_size=batch_size, num_workers=4
)


## 7.Definicja pętli treningowej:

In [ ]:
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for step, batch in tqdm(enumerate(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()

        total_loss += loss.item()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - avg_loss: {avg_loss:.4f}")
